# Information Extraction

#### Developed By: Manaranjan Pradhan
#### www.manaranjanp.com

*This Jupyter notebook is confidential and proprietary to Manaranjan Pradhan. It is intended solely for authorized training purposes. Unauthorized distribution, sharing, or reproduction of this notebook or its contents is strictly prohibited. This material is for personal learning within the training program only and may not be used for commercial purposes or shared with others. Unauthorized use may result in disciplinary action or legal consequences. If you have received this notebook without authorization, please contact manaranjan@gmail.com immediately and delete all copies.*



In [1]:
!pip install -q langchain openai langchain_openai langchain_openai langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.4 MB/s eta 0:00:00


[Source: Washington Post](https://www.washingtonpost.com/climate-environment/2023/04/11/electric-vehicle-buying-guide/)

## Accesing the Page Content

In [3]:
from bs4 import BeautifulSoup

with open("/content/zigwheels.html") as fp:
    soup = BeautifulSoup(fp, 'html.parser')

In [4]:
text = soup.get_text()

In [9]:
print(text[500:1000])

All Car Brands



Popular Car Models

 Best Cars in India »
 Maruti Grand Vitara
 Renault Kiger
 Skoda Slavia
 Tata Punch
 Hyundai Exter
 Maruti FRONX
 Maruti Swift
 Jeep Wrangler
 Jeep Compass
 Mahindra Thar
 Volkswagen Tiguan
 Mahindra Scorpio N
 Toyota Innova Hycross



Cars By Budget

Cars Under 5 Lakhs
Cars Under 8 Lakhs
Cars Under 10 Lakhs
Cars Under 15 Lakhs
Cars Under 20 Lakhs


Upcoming Cars
New Launches
Luxury Cars
Compare Cars
Car Dealers

CarDekho Services

Car Service History
Vahan 


In [10]:
beginning = 'Popular Electric Cars in India'
ending = "Key Highlights of Electric Cars"
final_text = text[text.index(beginning):text.index(ending)]
final_text = final_text.replace("\n", "\t")

In [11]:
print(final_text)

Popular Electric Cars in India							3.6	|  119 reviews						Tata Nexon EV		Rs. 14.49 Lakh		 465 km range ● 4 hour 20 min charging time ● 150 kmph	View March Offers	Check On-Road Price »									4.3	|  59 reviews						Tata Punch EV		Rs. 10.98 Lakh		 421 km range ● 56 min charging time ● 60kW	View March Offers	Check On-Road Price »									4.6	|  215 reviews						Tata Tiago EV		Rs. 7.99 Lakh		 315 km range ● 58 min charging time ● 45kW	View March Offers	Check On-Road Price »									4.1	|  63 reviews						Kia EV6		Rs. 60.95 Lakh		 708 km range ● 18 min charging time ● 192 kmph	View March Offers	Check On-Road Price »									3.7	|  139 reviews						MG Comet EV		Rs. 6.98 Lakh		 230 km range ● 3 hour 18 min charging time ● 41kW	View March Offers	Check On-Road Price »									5	|  read reviews						BYD Seal		Rs. 41.00 Lakh		 650 km range	View March Offers	Check On-Road Price »									5	|  43 reviews						BMW i7		Rs. 2.13 Crore		 625 km range ● 250 kmph ● 400kW	View March Offers	Check On-Ro

In [12]:
len(final_text)

5818

### Storing the page content in file

In [13]:
with open("ev_article.txt", "w") as f:
    f.write(final_text)

### Libararies or components used.

- OpenAI (https://platform.openai.com/)
- LangChain (https://langchain.com/)


## Configuring LLM for Query

In [14]:
import os
import openai
from getpass import getpass
from pprint import pprint

In [15]:
#os.environ['OPENAI_API_KEY'] = getpass('OPENAI_API_KEY')
os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


In [ ]:
#openai.api_key  = os.getenv('OPENAI_API_KEY')

### Initializing the OpenAI Chat Model

In [16]:
from langchain_openai import ChatOpenAI
from langchain.chains import create_extraction_chain, create_extraction_chain_pydantic
from langchain.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [17]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

# chat completion llm
#llm = ChatOpenAI(
#    model_name='gpt-4o-2024-11-20',
#    temperature=0.0
#)

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=2048,
)

### Configuring the Retrieval QA Chain

https://json-schema.org/understanding-json-schema/

In [18]:
schema = {
  'type': 'function',
  'function': {
    'name': 'CarListResponse',
    'description': '',
    'parameters': {
      'properties': {
        'cars': {
          'description': 'List of EV car details including model name, price, and range',
          'items': {
            'properties': {
              'car_model': {
                'description': 'The name of the EV model',
                'type': 'string'
              },
              'price': {
                'description': 'Price of the model in lakhs (Indian Rupees)',
                'type': 'string'
              },
              'range_kms': {
                'anyOf': [
                  {
                    'type': 'integer'
                  },
                  {
                    'type': 'null'
                  }
                ],
                'description': 'Kilometers it would travel after a single full charge'
              }
            },
            'required': [
              'car_model',
              'price',
              'range_kms'
            ],
            'type': 'object'
          },
          'type': 'array'
        }
      },
      'required': [
        'cars'
      ],
      'type': 'object'
    }
  }
}

### Configurating Conversational Memory

In [19]:
#chain = create_extraction_chain(schema, llm)
structured_llm = llm.with_structured_output(schema)

## Asking Questions

In [20]:
response = structured_llm.invoke(final_text)

In [21]:
print(response)

{'cars': [{'car_model': 'Tata Nexon EV', 'price': '14.49 Lakh', 'range_kms': 465}, {'car_model': 'Tata Punch EV', 'price': '10.98 Lakh', 'range_kms': 421}, {'car_model': 'Tata Tiago EV', 'price': '7.99 Lakh', 'range_kms': 315}, {'car_model': 'Kia EV6', 'price': '60.95 Lakh', 'range_kms': 708}, {'car_model': 'MG Comet EV', 'price': '6.98 Lakh', 'range_kms': 230}, {'car_model': 'BYD Seal', 'price': '41.00 Lakh', 'range_kms': 650}, {'car_model': 'BMW i7', 'price': '2.13 Crore', 'range_kms': 625}, {'car_model': 'Mahindra XUV400 EV', 'price': '15.49 Lakh', 'range_kms': 456}, {'car_model': 'MG ZS EV', 'price': '18.98 Lakh', 'range_kms': 461}, {'car_model': 'Hyundai Kona Electric', 'price': '23.84 Lakh', 'range_kms': 452}, {'car_model': 'BMW iX', 'price': '1.21 Crore', 'range_kms': 425}, {'car_model': 'Rolls-Royce Spectre', 'price': '7.50 Crore', 'range_kms': 530}, {'car_model': 'Tata Tigor EV', 'price': '12.49 Lakh', 'range_kms': 315}, {'car_model': 'BYD Atto 3', 'price': '33.99 Lakh', 'rang

### To buy or to wait

In [22]:
import pandas as pd
import json

In [23]:
ev_df = pd.DataFrame(response['cars'])

In [24]:
ev_df

,car_model,price,range_kms
0,Tata Nexon EV,14.49 Lakh,465.0
1,Tata Punch EV,10.98 Lakh,421.0
2,Tata Tiago EV,7.99 Lakh,315.0
3,Kia EV6,60.95 Lakh,708.0
4,MG Comet EV,6.98 Lakh,230.0
5,BYD Seal,41.00 Lakh,650.0
6,BMW i7,2.13 Crore,625.0
7,Mahindra XUV400 EV,15.49 Lakh,456.0
8,MG ZS EV,18.98 Lakh,461.0
9,Hyundai Kona Electric,23.84 Lakh,452.0


In [25]:
ev_df.to_csv('ev_models.csv', index = False)

## Using Pydantic

https://docs.pydantic.dev/latest/



In [26]:
from typing import Optional, List
from pydantic import BaseModel, Field

In [27]:
from pydantic import BaseModel, Field, validator

class CarDetails(BaseModel):
    car_model: str = Field(description="The name of the EV model")
    price: str = Field(description="Price of the model in lakhs (Indian Rupees)")
    range_kms: Optional[int] = Field(description="Kilometers it would travel after a single full charge")

class CarListResponse(BaseModel):
    cars: List[CarDetails] = Field(description="List of EV car details including model name, price, and range")

In [28]:
structured_llm = llm.with_structured_output(CarListResponse)

In [29]:
response = structured_llm.invoke(final_text)

In [30]:
print(response.dict()['cars'])

[{'car_model': 'Tata Nexon EV', 'price': '14.49', 'range_kms': 465}, {'car_model': 'Tata Punch EV', 'price': '10.98', 'range_kms': 421}, {'car_model': 'Tata Tiago EV', 'price': '7.99', 'range_kms': 315}, {'car_model': 'Kia EV6', 'price': '60.95', 'range_kms': 708}, {'car_model': 'MG Comet EV', 'price': '6.98', 'range_kms': 230}, {'car_model': 'BYD Seal', 'price': '41.00', 'range_kms': 650}, {'car_model': 'BMW i7', 'price': '2.13', 'range_kms': 625}, {'car_model': 'Mahindra XUV400 EV', 'price': '15.49', 'range_kms': 456}, {'car_model': 'MG ZS EV', 'price': '18.98', 'range_kms': 461}, {'car_model': 'Hyundai Kona Electric', 'price': '23.84', 'range_kms': 452}, {'car_model': 'BMW iX', 'price': '1.21', 'range_kms': 425}, {'car_model': 'Rolls-Royce Spectre', 'price': '7.50', 'range_kms': 530}, {'car_model': 'Tata Tigor EV', 'price': '12.49', 'range_kms': 315}, {'car_model': 'BYD Atto 3', 'price': '33.99', 'range_kms': 521}, {'car_model': 'BMW i4', 'price': '72.50', 'range_kms': 590}, {'car_m

<ipython-input-30-72d38f1110bc>:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  print(response.dict()['cars'])


In [31]:
structured_llm.to_json()

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'runnable', 'RunnableSequence'],
 'kwargs': {'first': RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7cc5598694d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7cc55983ebd0>, model_name='llama-3.3-70b-versatile', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********'), max_tokens=2048), kwargs={'tools': [{'type': 'function', 'function': {'name': 'CarListResponse', 'description': '', 'parameters': {'properties': {'cars': {'description': 'List of EV car details including model name, price, and range', 'items': {'properties': {'car_model': {'description': 'The name of the EV model', 'type': 'string'}, 'price': {'description': 'Price of the model in lakhs (Indian Rupees)', 'type': 'string'}, 'range_kms': {'anyOf': [{'type': 'integer'}, {'type': 'null'}], 'description': 'Kilometers it would travel after a single full charge'}}, 'req

In [32]:
pd.DataFrame(response.dict()['cars'])[0:10]

<ipython-input-32-8d14ed05dca6>:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  pd.DataFrame(response.dict()['cars'])[0:10]


,car_model,price,range_kms
0,Tata Nexon EV,14.49,465.0
1,Tata Punch EV,10.98,421.0
2,Tata Tiago EV,7.99,315.0
3,Kia EV6,60.95,708.0
4,MG Comet EV,6.98,230.0
5,BYD Seal,41.00,650.0
6,BMW i7,2.13,625.0
7,Mahindra XUV400 EV,15.49,456.0
8,MG ZS EV,18.98,461.0
9,Hyundai Kona Electric,23.84,452.0
